In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

# ---- STEP 1: Define ticker and expiry ----
ticker_symbol = "AAPL"   # you can change this
ticker = yf.Ticker(ticker_symbol)

# Get available expiration dates
expirations = ticker.options
print("Available Expirations:")
print(expirations)

# Choose a specific expiration (e.g., 3rd one)
expiry = expirations[2]
print(f"\nUsing expiration: {expiry}")

# ---- STEP 2: Get options chain ----
opt_chain = ticker.option_chain(expiry)
calls = opt_chain.calls
puts = opt_chain.puts

# ---- STEP 3: Extract strikes and implied vols ----
calls = calls[['strike', 'impliedVolatility']]
puts = puts[['strike', 'impliedVolatility']]

# Combine and clean
calls['type'] = 'call'
puts['type'] = 'put'
df = pd.concat([calls, puts])
df = df.dropna()

# ---- STEP 4: Plot Volatility Skew ----
plt.figure(figsize=(10, 6))
for opt_type, color in [('call', 'blue'), ('put', 'red')]:
    subset = df[df['type'] == opt_type]
    plt.plot(subset['strike'], subset['impliedVolatility'], label=f"{opt_type.capitalize()} IV", marker='o')

plt.title(f"Volatility Skew for {ticker_symbol} on {expiry}")
plt.xlabel("Strike Price")
plt.ylabel("Implied Volatility")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()